## Here, you will learn how to reshape your DataFrames using techniques such as pivoting, melting, stacking, and unstacking. These are powerful techniques that allow you to tidy and rearrange your data into the format that allows you to most easily analyze it for insights.

## Pivoting a single variable
Suppose you started a blog for a band, and you would like to log how many visitors you have had, and how many signed-up for your newsletter. To help design the tours later, you track where the visitors are. A DataFrame called users consisting of this information has been pre-loaded for you.

Inspect users in the IPython Shell and make a note of which variable you want to use to index the rows ('weekday'), which variable you want to use to index the columns ('city'), and which variable will populate the values in the cells ('visitors'). Try to visualize what the result should be  
In this exercise, your job is to pivot users so that the focus is on 'visitors', with the columns indexed by 'city' and the rows indexed by 'weekday'.

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
users = pd.read_csv('users.csv')
users.drop('Unnamed: 0', axis = 1, inplace = True)
users

,weekday,city,visitors,signups
0,Sun,Austin,139,7
1,Sun,Dallas,237,12
2,Mon,Austin,326,3
3,Mon,Dallas,456,5


In [5]:
# Pivot the users DataFrame with the rows indexed by 'weekday', the columns indexed by 'city', and the values populated with 'visitors'
visitors_pivot = users.pivot(index = 'weekday', columns = 'city', values = 'visitors')
visitors_pivot

city,Austin,Dallas
weekday,,
Mon,326,456
Sun,139,237


## Pivoting all variables
If you do not select any particular variables, all of them will be pivoted. In this case - with the users DataFrame - both 'visitors' and 'signups' will be pivoted, creating hierarchical column labels.

You will explore this for yourself now in this exercise.

In [6]:
# Pivot the users DataFrame with the 'signups' indexed by 'weekday' in the rows and 'city' in the columns.
users.pivot('weekday', 'city', 'signups')

city,Austin,Dallas
weekday,,
Mon,3,5
Sun,7,12


In [8]:
# Pivot the users DataFrame with both 'signups' and 'visitors' pivoted - that is, all the variables. This will happen automatically if you do not specify an argument for the values parameter of .pivot()
users_1 = users.pivot('weekday', 'city')
users_1

visitors        signups       
city      Austin Dallas  Austin Dallas
weekday                               
Mon          326    456       3      5
Sun          139    237       7     12

In [10]:
users_1_stack = users_1.stack(level = 'city')
users_1_stack

visitors  signups
weekday city                     
Mon     Austin       326        3
        Dallas       456        5
Sun     Austin       139        7
        Dallas       237       12

In [11]:
users_1_stack.swaplevel(0, 1)

,,visitors,signups
city,weekday,,
Austin,Mon,326,3
Dallas,Mon,456,5
Austin,Sun,139,7
Dallas,Sun,237,12


## Adding names for readability

In [14]:
visitors_by_city_weekday = visitors_pivot.reset_index()
visitors_by_city_weekday

city,weekday,Austin,Dallas
0,Mon,326,456
1,Sun,139,237


In [15]:
# Melt visitors_by_city_weekday: visitors
visitors = pd.melt(visitors_by_city_weekday, id_vars=['weekday'], value_name='visitors')

# Print visitors
visitors

,weekday,city,visitors
0,Mon,Austin,326
1,Sun,Austin,139
2,Mon,Dallas,456
3,Sun,Dallas,237


## Going from wide to long
You can move multiple columns into a single column (making the data long and skinny) by "melting" multiple columns. In this exercise, you will practice doing this.

The users DataFrame has been pre-loaded for you. As always, explore it in the IPython Shell and note the index.

In [16]:
users

,weekday,city,visitors,signups
0,Sun,Austin,139,7
1,Sun,Dallas,237,12
2,Mon,Austin,326,3
3,Mon,Dallas,456,5


In [17]:
# Define a DataFrame skinny where you melt the 'visitors' and 'signups' columns of users into a single column.
skinny = pd.melt(users, id_vars=['weekday', 'city'])
skinny

,weekday,city,variable,value
0,Sun,Austin,visitors,139
1,Sun,Dallas,visitors,237
2,Mon,Austin,visitors,326
3,Mon,Dallas,visitors,456
4,Sun,Austin,signups,7
5,Sun,Dallas,signups,12
6,Mon,Austin,signups,3
7,Mon,Dallas,signups,5


In [18]:
users_idx = users.set_index(['city', 'weekday'])
users_idx

,,visitors,signups
city,weekday,,
Austin,Sun,139,7
Dallas,Sun,237,12
Austin,Mon,326,3
Dallas,Mon,456,5


In [19]:
# Obtain the key-value pairs corresponding to visitors and signups by melting users_idx with the keyword argument col_level=0.
kv_pairs = pd.melt(users_idx, col_level=0)
kv_pairs


,variable,value
0,visitors,139
1,visitors,237
2,visitors,326
3,visitors,456
4,signups,7
5,signups,12
6,signups,3
7,signups,5


## Setting up a pivot table

In [21]:
users

,weekday,city,visitors,signups
0,Sun,Austin,139,7
1,Sun,Dallas,237,12
2,Mon,Austin,326,3
3,Mon,Dallas,456,5


In [20]:
# Use a pivot table to index the rows of users by 'weekday' and the columns of users by 'city'. These correspond to the index and columns parameters of .pivot_table().
by_city_day = users.pivot_table(index = 'weekday', columns = 'city')
by_city_day

signups        visitors       
city     Austin Dallas   Austin Dallas
weekday                               
Mon           3      5      326    456
Sun           7     12      139    237

## Using other aggregations in pivot tables
You can also use aggregation functions within a pivot table by specifying the aggfunc parameter. In this exercise, you will practice using the 'count' and len aggregation functions - which produce the same result - on the users DataFrame.

In [23]:
from IPython import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [24]:
# Use a pivot table to display the count of each column: count_by_weekday1
count_by_weekday1 = users.pivot_table(index ='weekday', aggfunc='count')

# Print count_by_weekday
count_by_weekday1

# Replace 'aggfunc='count'' with 'aggfunc=len': count_by_weekday2
count_by_weekday2 = users.pivot_table(index = 'weekday', aggfunc = len)

count_by_weekday2

# Verify that the same result is obtained
print('==========================================')
count_by_weekday1.equals(count_by_weekday2)

,city,signups,visitors
weekday,,,
Mon,2,2,2
Sun,2,2,2


,city,signups,visitors
weekday,,,
Mon,2,2,2
Sun,2,2,2


True

## Using margins in pivot tables
Sometimes it's useful to add totals in the margins of a pivot table. You can do this with the argument margins=True. In this exercise, you will practice using margins in a pivot table along with a new aggregation function: sum.

In [25]:
# Create the DataFrame with the appropriate pivot table: signups_and_visitors
signups_and_visitors = users.pivot_table(index='weekday', aggfunc=sum)

# Print signups_and_visitors
signups_and_visitors

# Add in the margins: signups_and_visitors_total 
signups_and_visitors_total = users.pivot_table(index='weekday', aggfunc=sum, margins=True)

# Print signups_and_visitors_total
signups_and_visitors_total

,signups,visitors
weekday,,
Mon,8,782
Sun,19,376


,signups,visitors
weekday,,
Mon,8,782
Sun,19,376
All,27,1158
